
# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

Implement a mini batch functionality to train a regressor.

(Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html
Save model, load the model again and test it on X_test Do NOT commit the pickle file

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)
    
    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df
    
    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
   
    while(True):
        yield df.sample(100).reset_index(drop=True)
        
gen = partial_df()

In [3]:
X_test, y_test = test_df()

In [4]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2006,Mazda,MX-5 Miata,Base,Convertible,manual,jm1nc25f960109815,tx,1.7,61473.0,gray,black,td auto finance,8225,1600,Wed May 06 2015 03:00:00 GMT-0700 (PDT)
1,2012,Infiniti,G Sedan,G37 Journey,g sedan,automatic,jn1cv6ap9cm626482,ga,4.1,31550.0,gray,gray,nissan infiniti lt,18600,18400,Tue Jun 16 2015 02:30:00 GMT-0700 (PDT)
2,2014,Kia,Optima,LX,Sedan,automatic,5xxgm4a75eg340269,nv,4.8,15878.0,silver,tan,"kia motors america, inc",14750,14900,Fri Jun 05 2015 05:00:00 GMT-0700 (PDT)
3,2013,Ford,Explorer,Limited,SUV,automatic,1fm5k8f81dgc72657,co,3.8,52992.0,gray,black,ars/avis budget group,26000,25900,Wed Feb 18 2015 03:00:00 GMT-0800 (PST)
4,2009,Chevrolet,Tahoe Hybrid,Base,SUV,NaN,1gnfk135x9r161179,nj,2.9,166770.0,black,black,pine belt enterprises inc,15250,11500,Wed Jan 21 2015 01:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2013,BMW,1 Series,128i,Convertible,automatic,wbaul7c59dvm83843,ca,3.9,15025.0,black,black,financial services remarketing (lease),24900,25500,Thu Feb 26 2015 04:30:00 GMT-0800 (PST)
96,2005,Buick,LaCrosse,CXL,Sedan,automatic,2g4wd532051196585,ga,2.3,152322.0,silver,black,wells fargo dealer services,3350,2600,Thu Jan 29 2015 02:00:00 GMT-0800 (PST)
97,2011,Dodge,Journey,Mainstreet,suv,automatic,3d4pg1fg2bt521265,ca,4.3,47093.0,white,black,american road leasing inc,12100,12250,Wed May 27 2015 05:15:00 GMT-0700 (PDT)
98,2001,BMW,7 Series,740i,Sedan,automatic,wbagg83421dn83962,ca,3.4,154484.0,gray,gray,auto depot,2575,1800,Thu Feb 05 2015 04:00:00 GMT-0800 (PST)


Importing Libraries

In [5]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

spliting features into numerical and categorical

In [6]:
num_features = list(X_test.select_dtypes(include='number').columns)
cat_features = list(X_test.select_dtypes(include=['object', 'category']).columns)
print(num_features)
print(cat_features)

['year', 'odometer']
['make', 'model', 'trim', 'body', 'transmission', 'vin', 'state', 'condition', 'color', 'interior', 'seller', 'mmr', 'saledate']


creating pipeline for numerical features 

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


creating pipeline for categorical features 

In [8]:
from sklearn.preprocessing import OneHotEncoder

categorical_pipeline = Pipeline([
    ('simpleimputer', SimpleImputer(strategy='most_frequent')),
    ('onehotencoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore'))
])

combining categorical and numerical pipelines

In [9]:
from sklearn.compose import ColumnTransformer

preprocessing_pipeline = ColumnTransformer([
    ('num', numerical_pipeline, num_features),
    ('cat', categorical_pipeline, cat_features)
])


preprocessing_pipeline.fit(X_test)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['year', 'odometer']),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['make', 'model', 'trim', 'body',
                                  'transmission', 'vin', 'state', 'condition',
                                  'color', 'interior', 'seller', 'mmr',
                                  'saledate'])])

Usage of SGDRegressor

In [13]:
regressor_batch_SGD = SGDRegressor(loss='squared_error', random_state=42)

Taken range of bathes is 55 and n as 100

In [15]:
batches = 55
n = 100
for i in range(n):
    for batch_index in range(batches):
        batch_df = next(gen)
        X_batch = batch_df.drop('sellingprice', axis=1)
        Y_batch = batch_df['sellingprice']

        X_batch_preprocessor = preprocessing_pipeline.transform(X_batch)

        regressor_batch_SGD.partial_fit(X_batch_preprocessor, Y_batch)


In [17]:
import joblib

joblib.dump(regressor_batch_SGD, 'regressor.joblib')
regressor = joblib.load('regressor.joblib')
X_test_processed = preprocessing_pipeline.transform(X_test)
y_pred = regressor.predict(X_test_processed)
print(y_pred)

[13115.33167042 19981.69595055 15543.23862733 ... 19884.03166698
 19456.81048427 15901.33297277]


Trained regressor model using joblib.dump(regressor_batch_SGD, 'regressor.joblib').
Load the saved model into a variable using regressor = joblib.load('regressor.joblib').
Preprocess the test data using a preprocessing pipeline or preprocessor object, assuming it is named preprocessing_pipeline. Apply the transformation to the test data with X_test_processed = preprocessing_pipeline.transform(X_test).
Use the loaded model to make predictions on the preprocessed test data: y_pred = regressor.predict(X_test_processed).
Print the predicted values using print(y_pred)

Printing the score

In [18]:
# Score
score = regressor.score(X_test_processed, y_test)
print("Model score:", score)

Model score: 0.8132016757774726


Accuracy of model is 81% with the use of SGDRegressor.